<a href="https://colab.research.google.com/github/sameeragrawal536/PyTorch-Neural-Network/blob/main/MNIST_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision.datasets as datasets
import time
import math

In [ ]:
transformer = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

#Loading training and testing data
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform = transformer)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform = transformer)

In [ ]:
class Net(nn.Module):
  #Architecture taken from Sentdex PyTorch tutorial
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 3)
    self.maxPool1 = nn.MaxPool2d((2, 2))
    self.dropout1 = nn.Dropout(0.5)
    self.fc1 = nn.Linear(13*13*32, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)
  
  #Softmax at the end to scale values to sum to 1
  def forward(self, input):
    first = torch.relu(self.conv1(input))
    small = self.maxPool1(first)
    #print(small.shape)
    droppedout = self.dropout1(small)
    #print(droppedout.shape)
    droppout2 = droppedout.view(-1, 13*13*32)
    l1 = torch.relu(self.fc1(droppout2))                
    l2 = torch.relu(self.fc2(l1))
    output = torch.softmax((self.fc3(l2)), dim = 1)
    return torch.log(output + 1e-10)

In [ ]:
neuralNetwork = Net()

#Creating train/test set
trainloader = torch.utils.data.DataLoader(mnist_trainset, batch_size=1, shuffle=True)
testloader = torch.utils.data.DataLoader(mnist_testset, batch_size=1, shuffle=True)

#This loss was used because labels are not 1 hot encoded
losscalc = nn.NLLLoss()
optimizer = optim.Adam(neuralNetwork.parameters(), lr = 0.001)

In [ ]:
for i in range(15):
  accuracy = 0
  j = 0
  for data in trainloader:
    inputs, labels = data
    #print(labels.shape)
    #print(inputs.shape)
    optimizer.zero_grad()
    #Reshaping data, flattening it
    #print(inputs.shape)
    outputs = neuralNetwork(inputs)
    #print(outputs.shape)
   # print(outputs.shape)
    loss = losscalc(outputs, labels)

    #Backpropogation + Gradient Descent
    loss.backward()
    optimizer.step()
    index = 0
    for i in range(10):
      if outputs[0][index].item() < outputs[0][i].item():
        index = i
    if index == labels[0].item():
      accuracy += 1
    j += 1
    #print(j)
  print("loss " + str(math.exp(loss)))
  print("accuracy " + str(accuracy/float(j)))

print("Done training!")